In [1]:
# import libraries

import os
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import mygene
import sklearn.preprocessing as sk
import seaborn as sns
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score, precision_recall_curve
import random
from random import randint
from sklearn.model_selection import StratifiedKFold
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
# define directories

cell_line_dir = "/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC/"
models_dir = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_DNA_REPi/Expression_CNA/Models/"
DEGs_dir = "/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/DNA_REPi/"
dataset_dir = "/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/"
save_results_to = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_DNA_REPi/Expression_CNA/Predictions/"

In [3]:
# set random seeds

torch.manual_seed(42)
random.seed(42)

In [4]:
# change directory to read GDSC cell line details

os.chdir(cell_line_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC


In [5]:
# read GDSC cell line details table

GDSC_cell_line_details = pd.read_excel("GDSC_Cell_Lines_Details.xlsx", keep_default_na = False)
GDSC_cell_line_details.set_index("COSMIC identifier", inplace = True)
GDSC_cell_line_details.tail()

,Sample Name,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
COSMIC identifier,,,,,,,,,,,,
1298218,KP-2,Y,N,Y,N,Y,pancreas,pancreas,PAAD,,D/F12,Adherent
1330932,KO52,Y,Y,N,N,N,leukemia,acute_myeloid_leukaemia,LAML,MSS/MSI-L,D/F12,NA
1331030,SC-1,Y,Y,N,N,N,lymphoma,B_cell_lymphoma,DLBC,MSS/MSI-L,R,NA
1503373,U-CH2,Y,Y,N,N,N,bone,bone_other,,MSS/MSI-L,D/F12,Adherent
,TOTAL:,1001,996,968,957,990,,,,,,


In [6]:
# change directory to read GDSC expression dataset (DNA_REPi)

os.chdir(dataset_dir + "/exprs/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs


In [7]:
# read GDSC expression dataset (DNA_REPi)

GDSCE = pd.read_csv("GDSC_exprs.z.DNA_REPi.tsv", 
                    sep = "\t", index_col=0, decimal = ",")
GDSCE = pd.DataFrame.transpose(GDSCE)
GDSCE.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,107984199,107984208,107984325,107984411,107984648,107984889,107984989,107986313,107986782,107986810
683665,-0.4711563767951986,-0.1857522957766964,0.9702409548662462,-0.4081281805960821,-0.4161511169999803,-0.4384159723467665,-1.154692246005183,0.5262132484107975,-0.7950924576033422,-0.6337442716864214,...,1.283534690590172,-0.3367265464586691,-0.0039762988723447,0.7378729872604209,0.9757614264575688,0.1494417647095801,2.062294063911948,-0.8140636761719959,0.2870333828833719,0.9524265085537148
684055,1.3565140948247052,-0.277542411913307,0.0887087882196536,-0.2754829982302413,-0.5153856346587746,-0.6248712076433911,0.1243408990866293,-1.2699667684674136,2.0788360886382717,2.5197991997459197,...,-0.5571028630154293,-1.724090733574364,-0.3749080365068901,-0.0016055431935905,-0.3788535663547365,-1.3502092798315848,0.6949790577550328,0.4333032198982747,-0.1280986270308099,-1.629249958712223
684057,0.6510004960254864,1.6578876382433665,-0.4977106390881842,-0.1212172092933644,-0.398455281697677,-0.2804901106672752,0.6455898116420014,0.215833091774836,-0.1723595788875009,-0.2170870770552223,...,-0.024863133395904,0.4073439017665206,-0.4440356638831038,0.8630347078663075,1.4486171428489227,-1.4777214830771732,0.5220400480464272,0.6441211958947753,0.9057518606131394,0.3018783753479829


In [8]:
# change directory to read GDSC CNA dataset (DNA_REPi)

os.chdir(dataset_dir + "/CNA/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/CNA


In [9]:
# read GDSC CNA dataset (DNA_REPi)

GDSCC = pd.read_csv("GDSC_CNA.DNA_REPi.tsv",
                    sep="\t", index_col=0, decimal=".")
GDSCC.drop_duplicates(keep='last')
GDSCC = pd.DataFrame.transpose(GDSCC)
GDSCC = GDSCC.loc[:, ~GDSCC.columns.duplicated()]

GDSCC.head()

gene_id,1,2,9,10,12,13,14,15,16,18,...,107985535,107985759,107986588,107986809,107986898,107987337,107987341,109731405,112441434,121676927
683665,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,1.807355,0.000000,0.000000,0.000000
684055,0.000000,0.584963,0.584963,0.584963,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.584963,0.000000,0.584963,0.584963,-4.32,-4.320000,0.000000,0.000000,0.584963
684057,-0.415037,0.000000,0.584963,0.584963,-1.000000,0.000000,0.0,0.0,-0.415037,-0.415037,...,-0.415037,0.000000,0.000000,0.000000,0.807355,-1.00,-1.000000,-0.415037,-0.415037,0.807355
684059,0.000000,0.000000,0.000000,0.000000,0.000000,0.415037,0.0,0.0,0.000000,0.415037,...,0.000000,0.000000,0.415037,0.000000,0.415037,-4.32,-4.320000,0.000000,0.000000,0.736966
684062,-0.415037,0.000000,0.321928,0.321928,-0.415037,-0.415037,0.0,0.0,-0.415037,0.000000,...,-0.415037,0.000000,0.000000,0.000000,0.321928,-4.32,-4.320000,-0.415037,-1.000000,0.321928


In [10]:
# change directory to read GDSC response dataset (DNA_REPi)

os.chdir(dataset_dir + "/response/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response


In [11]:
# read GDSC response dataset (DNA_REPi)

GDSCR = pd.read_csv("GDSC_response.DNA_REPi.tsv",
                    sep="\t",
                    index_col=0,
                    decimal=",")
GDSCR.dropna(inplace=True)
GDSCR.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
GDSCR["response"] = GDSCR.loc[:, "response"].apply(lambda x: d[x])

GDSCR.head()

,response,logIC50,drug,exprs,CNA,mutations
sample_name,,,,,,
683665,0,5.12075503641523,Pyrimethamine,1,1,1
683665,0,-3.03711635474651,Doxorubicin,1,1,1
683665,0,-0.712119156515401,Etoposide,1,1,1
683665,0,-4.40897202584936,Gemcitabine,1,1,1
683665,0,-1.59752443691324,Mitomycin C,1,1,1


In [12]:
# print count of cell lines for each drug (DNA_REPi)

GDSCR["drug"].value_counts()

drug
Etoposide         775
SN-38             771
Mitomycin C       767
Doxorubicin       766
5-Fluorouracil    764
Gemcitabine       761
Bleomycin         759
Temozolomide      752
Cisplatin         738
Methotrexate      738
Camptothecin      737
Cytarabine        735
Pyrimethamine     372
Name: count, dtype: int64

In [13]:
# change directory to read TCGA expression dataset homogenized with GDSC expression dataset (Gemcitabine)

os.chdir(dataset_dir + "/exprs_homogenized/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs_homogenized


In [14]:
# read TCGA expression dataset homogenized with GDSC expression dataset (Gemcitabine)

TCGAEgemcitabine = pd.read_csv("TCGA_exprs.Gemcitabine.eb_with.GDSC_exprs.Gemcitabine.tsv",
                   sep = "\t", index_col=0, decimal = ",")
TCGAEgemcitabine = pd.DataFrame.transpose(TCGAEgemcitabine)
TCGAEgemcitabine.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,100188893,100190940,100190949,100191040,100192386,100268168,100271715,100287718,100288778,100289635
TCGA-2L-AAQA-01A,3.98033768533371,2.62820652008909,7.03249429468481,3.08514834183615,3.34557990074588,3.78552975984518,8.73764964752829,2.48671779353536,8.21923206815042,3.76684655565602,...,10.0875725194276,2.81337967290836,3.51332685491118,3.98785715416424,3.07567558045391,2.56126348424232,2.63693987323012,2.83133300992887,5.13739151604047,4.69382514336529
TCGA-2L-AAQE-01A,4.68614471637408,3.25092419132046,5.75542155466148,2.79866479035723,1.96329083991897,3.43063349827168,8.80671304478463,2.62779485919227,8.54483025692787,3.81532940417382,...,8.87561409312678,2.81337967290836,2.78991865702044,3.30688425304558,3.45865630140412,2.53490145426966,2.74899279856495,2.83133300992887,4.61757632178081,4.2235154535305
TCGA-2L-AAQI-01A,3.71023111103364,3.88843437623036,5.77580575110606,3.10749843796475,5.98690471000489,3.75736553725429,8.4255369043372,2.48671779353536,9.21052968908206,3.97685914269242,...,9.48642956588486,2.84555917176674,2.78991865702044,3.48989111572046,2.79620231354257,2.49624245190844,2.630937590025,2.83133300992887,4.38859435268046,3.64555981820771


In [15]:
# change directory to read TCGA CNA dataset (Gemcitabine)

os.chdir(dataset_dir + "CNA")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/CNA'

In [16]:
# read TCGA CNA dataset (Gemcitabine)

TCGACgemcitabine = pd.read_csv("TCGA_CNA.Gemcitabine.tsv",
                        sep="\t", index_col=0, decimal=",")
TCGACgemcitabine.drop_duplicates(keep='last')
TCGACgemcitabine = pd.DataFrame.transpose(TCGACgemcitabine)
TCGACgemcitabine = TCGACgemcitabine.loc[:, ~TCGACgemcitabine.columns.duplicated()]

TCGACgemcitabine.head(3)

,13,24,30,34,39,55,68,73,86,93,...,100616290,100616357,100616366,100616420,100616434,100652770,100652778,100652797,100652854,100652917
TCGA-2L-AAQA-01A,0.0,-0.5075,0.0,-0.5075,-0.5104,0.0,-0.5104,0.0,0.0,0.0,...,-0.521,-0.521,-0.521,-0.521,0.0,-0.521,-0.521,-0.521,0.0,-0.521
TCGA-2L-AAQE-01A,0.0,0.0,0.0,0.0,-0.3378,0.0,-0.3346,0.0,0.0,0.0,...,-0.6335,-0.6335,-0.6409,-0.6409,0.0,-0.6409,-0.6409,-0.6409,0.0,-0.6633
TCGA-2L-AAQI-01A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# variance threshold for GDSC expression dataset (DNA_REPi)

selector = VarianceThreshold(0.05)
selector.fit_transform(GDSCE)
GDSCE = GDSCE[GDSCE.columns[selector.get_support(indices=True)]]

In [18]:
# fill NA values and binarize GDSC CNA dataset (DNA_REPi)

GDSCC = GDSCC.fillna(0)
GDSCC[GDSCC != 0.0] = 1

In [19]:
# select shared genes between GDSC and TCGA expression and CNA datasets 

ls = GDSCE.columns.intersection(GDSCC.columns)
ls = ls.intersection(TCGAEgemcitabine.columns)
ls = ls.intersection(TCGACgemcitabine.columns)

ls = pd.unique(ls)

print(f"GDSC and TCGA # of common genes: {ls.shape[0]}")

GDSC and TCGA # of common genes: 16389


In [20]:
# select shared samples between GDSC expression and response datasets (DNA_REPi)

ls2 = GDSCE.index.intersection(GDSCC.index)
ls2 = ls2.intersection(GDSCR.index)

print(f"GDSC # of common cell lines: {ls2.shape[0]}")

GDSC # of common cell lines: 801


In [21]:
# subset shared genes and samples in GDSC expression, CNA, and response datasets

GDSCE = GDSCE.loc[ls2, ls]
GDSCC = GDSCC.loc[ls2, ls]
GDSCR = GDSCR.loc[ls2, :]

print(f"GDSC # of common samples and genes (Expression): {GDSCE.shape}")
print(f"GDSC # of common samples and genes (CNA): {GDSCC.shape}")
print(f"GDSC # of samples (Response): {GDSCR.shape[0]}")

# There exists same sample names for different drugs, so row shape is different in GDSC response data

GDSC # of common samples and genes (Expression): (801, 16389)
GDSC # of common samples and genes (CNA): (801, 16389)
GDSC # of samples (Response): 9435


In [22]:
# select shared samples between TCGA expression and CNA datasets (Gemcitabine) 

ls3 = TCGAEgemcitabine.index.intersection(TCGACgemcitabine.index)

print(f"TCGA # of common samples (Gemcitabine): {ls3.shape[0]}")

TCGA # of common samples (Gemcitabine): 55


In [23]:
# select shared genes and samples between TCGA expression and CNA datasets (Gemcitabine)

TCGAEgemcitabine = TCGAEgemcitabine.loc[ls3, ls]
TCGACgemcitabine = TCGACgemcitabine.loc[ls3, ls]

print(f"TCGA # of common samples and genes for Expression (Gemcitabine): {TCGAEgemcitabine.shape}")
print(f"TCGA # of common samples and genes for CNA (Gemcitabine): {TCGACgemcitabine.shape}\n")

TCGA # of common samples and genes for Expression (Gemcitabine): (55, 16389)
TCGA # of common samples and genes for CNA (Gemcitabine): (55, 16389)



In [24]:
# change directory to read DEGs (DNA_REPi)

os.chdir(DEGs_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/DNA_REPi


In [25]:
# read DEGs (DNA_REPi)

DEGs_filtered_data = pd.read_excel("DNA_REPi_Differentially_Expressed_Genes (EnsemblID).xlsx",
                                    sheet_name = "Common DEGs")
print(f"There are {DEGs_filtered_data.shape[0]} DEGs (p-adjusted < 0.05 and |logFC| > 1) among DNA replication inhibitors\n")

DEGs_filtered_data.head()

There are 858 DEGs (p-adjusted < 0.05 and |logFC| > 1) among DNA replication inhibitors



,Gene Symbol,Frequency
0,SLFN11,9
1,NCKAP1L,8
2,SDC4,7
3,KRT18,7
4,S100A14,7


In [26]:
# get gene symbol from mygene database

mg = mygene.MyGeneInfo()
DEGs_entrez_id = mg.querymany(DEGs_filtered_data["Gene Symbol"], 
                              species="human", 
                              scopes = "symbol",
                              field = "entrezgene",
                              as_dataframe=True)["entrezgene"]
DEGs_entrez_id = pd.unique(DEGs_entrez_id.dropna())
DEGs_entrez_id.shape[0]

1 input query terms found dup hits:	[('CLECL1P', 2)]


858

In [27]:
# assign GDSC datasets to new variables

exprs_z = GDSCE
cna = GDSCC
responses = GDSCR

In [28]:
# GDSC response dataset

responses.head(3)

,response,logIC50,drug,exprs,CNA,mutations
683665,0,5.12075503641523,Pyrimethamine,1,1,1
683665,0,-3.03711635474651,Doxorubicin,1,1,1
683665,0,-0.712119156515401,Etoposide,1,1,1


In [29]:
# list drug names in GDSC response dataset

drugs = set(responses["drug"].values)
drugs

{'5-Fluorouracil',
 'Bleomycin',
 'Camptothecin',
 'Cisplatin',
 'Cytarabine',
 'Doxorubicin',
 'Etoposide',
 'Gemcitabine',
 'Methotrexate',
 'Mitomycin C',
 'Pyrimethamine',
 'SN-38',
 'Temozolomide'}

In [30]:
# convert Entrez gene IDs to integer

responses.index = responses.index.astype(int)
responses.index

Index([ 683665,  683665,  683665,  683665,  683665,  683665,  683665,  683665,
        683665,  683665,
       ...
       1659818, 1659818, 1659818, 1659818, 1659819, 1659819, 1659819, 1659819,
       1659819,  687800],
      dtype='int64', length=9435)

In [31]:
# merge GDSC cell line details with response dataset

dat = pd.merge(GDSC_cell_line_details,
         responses,
         left_index = True, 
         right_index=True,
         how = "inner")

dat["GDSC\nTissue descriptor 1"].value_counts()
#dat.head()

GDSC\nTissue descriptor 1
lung_NSCLC           1037
urogenital_system     926
leukemia              829
lymphoma              769
aero_dig_tract        727
nervous_system        607
lung_SCLC             569
skin                  560
breast                524
digestive_system      466
large_intestine       447
bone                  421
neuroblastoma         332
pancreas              294
kidney                227
soft_tissue           191
myeloma               182
lung                  174
thyroid               153
Name: count, dtype: int64

In [32]:
# list count of cell lines as to tissues treated with gemcitabine

filter = (dat["drug"] == "Gemcitabine")
dat.loc[filter,"GDSC\nTissue descriptor 1"].value_counts()

GDSC\nTissue descriptor 1
lung_NSCLC           86
urogenital_system    76
leukemia             64
aero_dig_tract       62
lymphoma             59
nervous_system       47
skin                 47
lung_SCLC            46
breast               43
digestive_system     38
large_intestine      35
bone                 34
neuroblastoma        25
pancreas             25
kidney               17
soft_tissue          16
lung                 15
myeloma              13
thyroid              13
Name: count, dtype: int64

In [33]:
# convert Entrez gene IDs to string

responses.index = responses.index.astype(str)
responses.index

Index(['683665', '683665', '683665', '683665', '683665', '683665', '683665',
       '683665', '683665', '683665',
       ...
       '1659818', '1659818', '1659818', '1659818', '1659819', '1659819',
       '1659819', '1659819', '1659819', '687800'],
      dtype='object', length=9435)

In [34]:
# filter GDSC expression and CNA datasets (DNA_REPi) as to drugs
# subset selected DEGs

expression_zscores = []
CNA = []
for drug in drugs:
    samples = responses.loc[responses["drug"] == drug, :].index.values
    e_z = exprs_z.loc[samples, :]
    c = cna.loc[samples, :]
    expression_zscores.append(e_z)
    CNA.append(c)

GDSCEv2 = pd.concat(expression_zscores, axis=0)
GDSCCv2 = pd.concat(CNA, axis=0)
GDSCRv2 = responses

ls4 = list(set(GDSCE.columns).intersection(set(DEGs_entrez_id.astype(int))))
ls5 = GDSCEv2.index.intersection(GDSCCv2.index)

GDSCEv2 = GDSCEv2.loc[ls5, ls4]
GDSCCv2 = GDSCCv2.loc[ls5, ls4]
GDSCRv2 = GDSCRv2.loc[ls5, :]

TCGAEgemcitabine = TCGAEgemcitabine.loc[:,ls4]
TCGACgemcitabine = TCGACgemcitabine.loc[:,ls4]

responses.index = responses.index.values + "_" + responses["drug"].values

print(f"GDSC # of common samples and genes (Expression): {GDSCEv2.shape}")
print(f"GDSC # of common samples and genes (CNA): {GDSCCv2.shape}")
print(f"GDSC # of common samples (Response): {GDSCRv2.shape[0]}\n")

print(f"TCGA # of common samples and genes for Gemcitabine (Expression): {TCGAEgemcitabine.shape}")
print(f"TCGA # of common samples and genes for Gemcitabine (CNA): {TCGACgemcitabine.shape}\n")

GDSC # of common samples and genes (Expression): (9435, 772)
GDSC # of common samples and genes (CNA): (9435, 772)
GDSC # of common samples (Response): 9435

TCGA # of common samples and genes for Gemcitabine (Expression): (55, 772)
TCGA # of common samples and genes for Gemcitabine (CNA): (55, 772)



In [35]:
# GDSC response values

Y = GDSCRv2['response'].values
print(Y)

[0 0 0 ... 0 0 1]


In [36]:
# change directory to read TCGA response dataset (Gemcitabine)

os.chdir(dataset_dir + "response")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response'

In [37]:
# read TCGA response dataset (Gemcitabine)

TCGARgemcitabineorubicin = pd.read_csv("TCGA_response.Gemcitabine.tsv", 
                      sep = "\t", 
                      index_col=0, 
                      decimal = ",")
TCGARgemcitabineorubicin.dropna(inplace=True)
TCGARgemcitabineorubicin.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
TCGARgemcitabineorubicin["response"] = TCGARgemcitabineorubicin.loc[:, "response"].apply(lambda x: d[x])
Ytsgemcitabine = TCGARgemcitabineorubicin["response"].values    

print(f"There are {Ytsgemcitabine.shape[0]} samples with response data in the TCGA (Gemcitabine)\n")
Ytsgemcitabine

There are 55 samples with response data in the TCGA (Gemcitabine)



array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0])

In [38]:
# TCGA cohort table with TCGA response dataset (Gemcitabine)

dat_tcga = TCGARgemcitabineorubicin["cohort"]

dat_tcga.value_counts()

cohort
PAAD    48
SARC     3
LIHC     2
LUSC     1
PCPG     1
Name: count, dtype: int64

In [39]:
# filter PAAD cohort samples (Gemcitabine)

filter = (TCGARgemcitabineorubicin["cohort"] == "PAAD")
tcga_paad_samples = dat_tcga.loc[filter,].index
tcga_paad_samples

Index(['TCGA-2L-AAQA-01A', 'TCGA-2L-AAQE-01A', 'TCGA-2L-AAQI-01A',
       'TCGA-2L-AAQJ-01A', 'TCGA-3A-A9I9-01A', 'TCGA-3A-A9IH-01A',
       'TCGA-3A-A9IX-01A', 'TCGA-3A-A9IZ-01A', 'TCGA-3A-A9J0-01A',
       'TCGA-F2-6879-01A', 'TCGA-F2-6880-01A', 'TCGA-F2-7273-01A',
       'TCGA-F2-7276-01A', 'TCGA-F2-A44G-01A', 'TCGA-F2-A44H-01A',
       'TCGA-F2-A8YN-01A', 'TCGA-FB-A5VM-01A', 'TCGA-FB-A78T-01A',
       'TCGA-FB-AAPP-01A', 'TCGA-FB-AAPY-01A', 'TCGA-FB-AAQ6-01A',
       'TCGA-H8-A6C1-01A', 'TCGA-HZ-7924-01A', 'TCGA-HZ-8317-01A',
       'TCGA-HZ-A49G-01A', 'TCGA-HZ-A49H-01A', 'TCGA-HZ-A77O-01A',
       'TCGA-HZ-A77P-01A', 'TCGA-IB-7644-01A', 'TCGA-IB-7647-01A',
       'TCGA-IB-7651-01A', 'TCGA-IB-7652-01A', 'TCGA-IB-7885-01A',
       'TCGA-IB-7889-01A', 'TCGA-IB-7891-01A', 'TCGA-IB-8127-01A',
       'TCGA-IB-A5SO-01A', 'TCGA-IB-A5SP-01A', 'TCGA-IB-A5SS-01A',
       'TCGA-IB-A6UF-01A', 'TCGA-IB-A7LX-01A', 'TCGA-IB-AAUO-01A',
       'TCGA-LB-A8F3-01A', 'TCGA-Q3-A5QY-01A', 'TCGA-RB-AA9M-0

In [40]:
# filter responses of PAAD cohort samples (Gemcitabine)

Ytsgemcitabine_paad = TCGARgemcitabineorubicin.loc[tcga_paad_samples,"response"].values
Ytsgemcitabine_paad

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0])

In [41]:
# count of sensitive and resistance PAAD samples (Gemcitabine)

TCGARgemcitabineorubicin.loc[tcga_paad_samples,"response"].value_counts()

response
0    28
1    20
Name: count, dtype: int64

In [42]:
# define maximum iteration

max_iter = 50

In [43]:
# assign number of neurons and dropout rates found for layers from the saved models

hdm1 = 64
hdm2 = 32
rate1 = 0.5
rate2 = 0.6
rate3 = 0.6

In [44]:
# load pre-trained models and make predictions

scalerGDSC = sk.StandardScaler()
scalerGDSC.fit(GDSCEv2.values)
X_trainE = scalerGDSC.transform(GDSCEv2.values)
X_testEgemcitabine = scalerGDSC.transform(TCGAEgemcitabine.values)    

X_testEgemcitabine = scalerGDSC.transform(TCGAEgemcitabine.values)    
X_testEgemcitabine_paad = scalerGDSC.transform(TCGAEgemcitabine.loc[tcga_paad_samples,].values)    

X_trainC = np.nan_to_num(GDSCCv2.values)
X_testCgemcitabine = np.nan_to_num(TCGACgemcitabine.values.astype(np.float32))
X_testCgemcitabine_paad = np.nan_to_num(TCGACgemcitabine.loc[tcga_paad_samples,].values)

TX_testEgemcitabine = torch.FloatTensor(X_testEgemcitabine)
TX_testCgemcitabine = torch.FloatTensor(X_testCgemcitabine.astype(np.float32))
ty_testEgemcitabineorouracil = torch.FloatTensor(Ytsgemcitabine.astype(int))

TX_testEgemcitabine = torch.FloatTensor(X_testEgemcitabine)
TX_testEgemcitabine_paad = torch.FloatTensor(X_testEgemcitabine_paad)

TX_testCgemcitabine = torch.FloatTensor(X_testCgemcitabine.astype(np.float32))
TX_testCgemcitabine_paad = torch.FloatTensor(X_testCgemcitabine_paad.astype(np.float32))

n_sampE, IE_dim = X_trainE.shape
n_sampC, IC_dim = X_trainC.shape

h_dim1 = hdm1
h_dim2 = hdm2
Z_in = h_dim1 + h_dim2 

class AEE(nn.Module):
    def __init__(self):
        super(AEE, self).__init__()
        self.EnE = torch.nn.Sequential(
            nn.Linear(IE_dim, h_dim1),
            nn.BatchNorm1d(h_dim1),
            nn.ReLU(),
            nn.Dropout(rate1))
    def forward(self, x):
        output = self.EnE(x)
        return output  


class AEC(nn.Module):
    def __init__(self):
        super(AEC, self).__init__()
        self.EnC = torch.nn.Sequential(
            nn.Linear(IC_dim, h_dim2),
            nn.BatchNorm1d(h_dim2),
            nn.ReLU(),
            nn.Dropout(rate2))
    def forward(self, x):
        output = self.EnC(x)
        return output       

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.FC = torch.nn.Sequential(
            nn.Linear(Z_in, 1),
            nn.Dropout(rate3),
            nn.Sigmoid())
    def forward(self, x):
        return self.FC(x)

torch.cuda.manual_seed_all(42)

AutoencoderE = torch.load(models_dir + "Exprs_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA.pt")
AutoencoderC = torch.load(models_dir + "CNA_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA.pt")

Clas = torch.load(models_dir + "Class_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA.pt")

AutoencoderE.eval()
AutoencoderC.eval()
Clas.eval()

ZEX = AutoencoderE(torch.FloatTensor(X_trainE))
ZCX = AutoencoderC(torch.FloatTensor(X_trainC))
ZTX = torch.cat((ZEX, ZCX), 1)
ZTX = F.normalize(ZTX, p=2, dim=0)
PredX = Clas(ZTX)
AUCT = roc_auc_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUC (TCGA_DNA_REPi): {round(AUCT,2)}")
AUCTaucpr = average_precision_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUCPR (TCGA_DNA_REPi): {round(AUCTaucpr,2)}\n")

ZETgemcitabine = AutoencoderE(TX_testEgemcitabine)
ZCTgemcitabine = AutoencoderC(TX_testCgemcitabine)
ZTTgemcitabine = torch.cat((ZETgemcitabine, ZCTgemcitabine), 1)
ZTTgemcitabine = F.normalize(ZTTgemcitabine, p=2, dim=0)
PredTgemcitabine = Clas(ZTTgemcitabine)
AUCTgemcitabine = roc_auc_score(Ytsgemcitabine.astype(int), PredTgemcitabine.detach().numpy())
#print(f"TCGA - Test AUC (Gemcitabine): {round(AUCTgemcitabine,2)}")
AUCTgemcitabine_aucpr = average_precision_score(Ytsgemcitabine.astype(int), PredTgemcitabine.detach().numpy())
#print(f"TCGA - Test AUCPR (Gemcitabine): {round(AUCTgemcitabine_aucpr,2)}\n")

ZETgemcitabine_paad = AutoencoderE(TX_testEgemcitabine_paad)
ZCTgemcitabine_paad = AutoencoderC(TX_testCgemcitabine_paad)
ZTTgemcitabine_paad = torch.cat((ZETgemcitabine_paad, ZCTgemcitabine_paad), 1)
ZTTgemcitabine_paad = F.normalize(ZTTgemcitabine_paad, p=2, dim=0)
PredTgemcitabine_paad = Clas(ZTTgemcitabine_paad)
AUCTgemcitabine_paad = roc_auc_score(Ytsgemcitabine_paad.astype(int), PredTgemcitabine_paad.detach().numpy())
#print(f"TCGA - Test AUC (Gemcitabine, PAAD): {round(AUCTgemcitabine_paad,2)}")
AUCTgemcitabine_paad_aucpr = average_precision_score(Ytsgemcitabine_paad.astype(int), PredTgemcitabine_paad.detach().numpy())
#print(f"TCGA - Test AUCPR (Gemcitabine, PAAD): {round(AUCTgemcitabine_paad_aucpr,2)}")

AUCT_finetuned = dict()
AUCTaucpr_finetuned = dict()
AUCTgemcitabine_finetuned = dict()
AUCTgemcitabine_paad_finetuned = dict()
AUCTgemcitabine_aucpr_finetuned = dict()
AUCTgemcitabine_paad_aucpr_finetuned = dict()
for iter in range(max_iter):
   iter += 1  

   AutoencoderE_finetuned = torch.load(models_dir + f"Finetuned_Models/Exprs_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA_Finetuned_{iter}.pt")
   AutoencoderC_finetuned = torch.load(models_dir + f"Finetuned_Models/CNA_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA_Finetuned_{iter}.pt")
   Clas_finetuned = torch.load(models_dir + f"Finetuned_Models/Class_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA_Finetuned_{iter}.pt")
   
   AutoencoderE_finetuned.eval()
   AutoencoderC_finetuned.eval()
   Clas_finetuned.eval()
      
   ZEX = AutoencoderE_finetuned(torch.FloatTensor(X_trainE))
   ZCX = AutoencoderC_finetuned(torch.FloatTensor(X_trainC))
   ZTX = torch.cat((ZEX, ZCX), 1)
   ZTX = F.normalize(ZTX, p=2, dim=0)
   PredX = Clas_finetuned(ZTX)
   AUCT_finetuned[iter] = roc_auc_score(Y, PredX.detach().numpy())
   AUCTaucpr_finetuned[iter] = average_precision_score(Y, PredX.detach().numpy())
   
   ZETgemcitabine = AutoencoderE_finetuned(TX_testEgemcitabine)
   ZCTgemcitabine = AutoencoderC_finetuned(TX_testCgemcitabine)
   ZTTgemcitabine = torch.cat((ZETgemcitabine, ZCTgemcitabine), 1)
   ZTTgemcitabine = F.normalize(ZTTgemcitabine, p=2, dim=0)
   PredTgemcitabine = Clas_finetuned(ZTTgemcitabine)
   AUCTgemcitabine_finetuned[iter] = roc_auc_score(Ytsgemcitabine.astype(int), PredTgemcitabine.detach().numpy())
   AUCTgemcitabine_aucpr_finetuned[iter] = average_precision_score(Ytsgemcitabine.astype(int), PredTgemcitabine.detach().numpy())

   ZETgemcitabine_paad = AutoencoderE_finetuned(TX_testEgemcitabine_paad)
   ZCTgemcitabine_paad = AutoencoderC_finetuned(TX_testCgemcitabine_paad)
   ZTTgemcitabine_paad = torch.cat((ZETgemcitabine_paad, ZCTgemcitabine_paad), 1)
   ZTTgemcitabine_paad = F.normalize(ZTTgemcitabine_paad, p=2, dim=0)
   PredTgemcitabine_paad = Clas_finetuned(ZTTgemcitabine_paad)
   AUCTgemcitabine_paad_finetuned[iter] = roc_auc_score(Ytsgemcitabine_paad.astype(int), PredTgemcitabine_paad.detach().numpy())
   AUCTgemcitabine_paad_aucpr_finetuned[iter] = average_precision_score(Ytsgemcitabine_paad.astype(int), PredTgemcitabine_paad.detach().numpy())

max_key = max(AUCTgemcitabine_aucpr_finetuned, key=AUCTgemcitabine_aucpr_finetuned.get)
max_key_paad = max(AUCTgemcitabine_paad_aucpr_finetuned, key=AUCTgemcitabine_paad_aucpr_finetuned.get)

print(f"GDSC - Training AUC (DNA_REPi): {round(AUCT,2)}")
print(f"GDSC - Training AUCPR (DNA_REPi): {round(AUCTaucpr,2)}")
print(f"TCGA - Test AUC (Gemcitabine): {round(AUCTgemcitabine,2)}")
print(f"TCGA - Test AUCPR (Gemcitabine): {round(AUCTgemcitabine_aucpr,2)}")
print(f"TCGA - Test AUC (Gemcitabine, PAAD): {round(AUCTgemcitabine_paad,2)}")
print(f"TCGA - Test AUCPR (Gemcitabine, PAAD): {round(AUCTgemcitabine_paad_aucpr,2)}\n")

print(f"GDSC - Training AUC (DNA_REPi, Finetuned): {round(AUCT_finetuned[max_key],2)}")
print(f"GDSC - Training AUCPR (DNA_REPi, Finetuned): {round(AUCTaucpr_finetuned[max_key],2)}")
print(f"TCGA - Test AUC (Gemcitabine, Finetuned): {round(AUCTgemcitabine_finetuned[max_key],2)}")
print(f"TCGA - Test AUCPR (Gemcitabine, Finetuned): {round(AUCTgemcitabine_aucpr_finetuned[max_key],2)}\n")

print(f"GDSC - Training AUC (DNA_REPi, Finetuned): {round(AUCT_finetuned[max_key_paad],2)}")
print(f"GDSC - Training AUCPR (DNA_REPi, Finetuned): {round(AUCTaucpr_finetuned[max_key_paad],2)}")
print(f"TCGA - Test AUC (Gemcitabine, PAAD, Finetuned): {round(AUCTgemcitabine_paad_finetuned[max_key_paad],2)}")
print(f"TCGA - Test AUCPR (Gemcitabine, PAAD, Finetuned): {round(AUCTgemcitabine_paad_aucpr_finetuned[max_key_paad],2)}\n\n")

print("Models for maximum finetuning (Gemcitabine)")
print(f"Exprs_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA_Finetuned_{max_key}.pt")
print(f"CNA_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA_Finetuned_{max_key}.pt")
print(f"Class_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA_Finetuned_{max_key}.pt")

print("\nModels for maximum finetuning (Gemcitabine, PAAD)")
print(f"Exprs_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA_Finetuned_{max_key_paad}.pt")
print(f"CNA_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA_Finetuned_{max_key_paad}.pt")
print(f"Class_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA_Finetuned_{max_key_paad}.pt")

GDSC - Training AUC (DNA_REPi): 0.75
GDSC - Training AUCPR (DNA_REPi): 0.27
TCGA - Test AUC (Gemcitabine): 0.6
TCGA - Test AUCPR (Gemcitabine): 0.41
TCGA - Test AUC (Gemcitabine, PAAD): 0.67
TCGA - Test AUCPR (Gemcitabine, PAAD): 0.63

GDSC - Training AUC (DNA_REPi, Finetuned): 0.76
GDSC - Training AUCPR (DNA_REPi, Finetuned): 0.27
TCGA - Test AUC (Gemcitabine, Finetuned): 0.62
TCGA - Test AUCPR (Gemcitabine, Finetuned): 0.43

GDSC - Training AUC (DNA_REPi, Finetuned): 0.76
GDSC - Training AUCPR (DNA_REPi, Finetuned): 0.27
TCGA - Test AUC (Gemcitabine, PAAD, Finetuned): 0.71
TCGA - Test AUCPR (Gemcitabine, PAAD, Finetuned): 0.69


Models for maximum finetuning (Gemcitabine)
Exprs_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA_Finetuned_43.pt
CNA_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA_Finetuned_43.pt
Class_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA_Finetuned_43.pt

Models for maximum finetuning (Gemcitabine, PAAD)
Exprs_Gemcitabine_GDSC_TCGA_DNA_REPi_Expression_CNA_Finetuned_13

In [51]:
# create a dataframe with the prediction results 

ec = {"Data": ["Expr + CNA", "Expr + CNA (Fine-tuned)"],
       "AUC (GDSC)": [round(AUCT, 2), round(AUCT_finetuned[max_key], 2)],
       "AUCPR (GDSC)": [round(AUCTaucpr,2), round(AUCTaucpr_finetuned[max_key],2)],
       "AUC (TCGA)": [round(AUCTgemcitabine,2), round(AUCTgemcitabine_finetuned[max_key],2)],
       "AUCPR (TCGA)":  [round(AUCTgemcitabine_aucpr,2), round(AUCTgemcitabine_aucpr_finetuned[max_key],2)],
       "AUC (TCGA-PAAD)":  [round(AUCTgemcitabine_paad,2), round(AUCTgemcitabine_paad_finetuned[max_key_paad],2)],
       "AUCPR (TCGA-PAAD)": [round(AUCTgemcitabine_paad_aucpr,2), round(AUCTgemcitabine_paad_aucpr_finetuned[max_key_paad],2)],
       "Sample Size (GDSC)": [GDSCEv2.shape[0], GDSCEv2.shape[0]],
       "Feature Size (GDSC)": [GDSCEv2.shape[1], GDSCEv2.shape[1]] 
}
ec_dataframe = pd.DataFrame.from_dict(ec, orient='index').transpose() 

ec_dataframe.to_csv(save_results_to + "GDSC_TCGA_Expression_CNA_DNA_REPi_Gemcitabine.tsv",
                    sep = "\t",
                    index = False)

ec_dataframe

,Data,AUC (GDSC),AUCPR (GDSC),AUC (TCGA),AUCPR (TCGA),AUC (TCGA-PAAD),AUCPR (TCGA-PAAD),Sample Size (GDSC),Feature Size (GDSC)
0,Expr + CNA,0.75,0.27,0.6,0.41,0.67,0.63,9435,772
1,Expr + CNA (Fine-tuned),0.76,0.27,0.62,0.43,0.71,0.69,9435,772


In [52]:
# show expression layer parameters

AutoencoderE

AEE(
  (EnE): Sequential(
    (0): Linear(in_features=772, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
  )
)

In [53]:
# show CNA layer parameters

AutoencoderC

AEC(
  (EnC): Sequential(
    (0): Linear(in_features=772, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.6, inplace=False)
  )
)

In [54]:
# show classification layer parameters

Clas

Classifier(
  (FC): Sequential(
    (0): Linear(in_features=96, out_features=1, bias=True)
    (1): Dropout(p=0.6, inplace=False)
    (2): Sigmoid()
  )
)

In [55]:
# change directory

os.chdir(save_results_to)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_DNA_REPi/Expression_CNA/Predictions


In [56]:
# save predictions

file = open('GDSC_TCGA_DEGs - TCGA_DNA_REPi_Gemcitabine Predictions (Expression and CNA).txt', 'w')
file.write(f"GDSC Training (EC) AUC (TCGA_DNA_REPi): {round(AUCT,2)}\n")
file.write(f"GDSC Training (EC) AUCPR (TCGA_DNA_REPi): {round(AUCTaucpr,2)}\n\n")

file.write(f"TCGA Test (EC) AUC (Gemcitabine): {round(AUCTgemcitabine,2)}\n")
file.write(f"TCGA Test (EC) AUCPR (Gemcitabine): {round(AUCTgemcitabine_aucpr,2)}\n\n")

file.close()